# Machine Learning Model (EMMA)

### This web application aims to tell you where you should roll your business out in Mexico, based on demographic information (income, location, education, and more)


# ETL process

In [2]:
pip install geopandas

  Running setup.py clean for fiona
Failed to build fiona
    Running setup.py install for fiona: started
    Running setup.py install for fiona: finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


    ERROR: Command errored out with exit status 1:
     command: 'D:\Users\rodrigo.guarneros\Anaconda4\python.exe' -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'D:\\Users\\rodrigo.guarneros\\AppData\\Local\\Temp\\pip-install-mha8smgt\\fiona_ab26aab3639e418c96b9756ea5b10a81\\setup.py'"'"'; __file__='"'"'D:\\Users\\rodrigo.guarneros\\AppData\\Local\\Temp\\pip-install-mha8smgt\\fiona_ab26aab3639e418c96b9756ea5b10a81\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base 'D:\Users\rodrigo.guarneros\AppData\Local\Temp\pip-pip-egg-info-xd1zls_6'
         cwd: D:\Users\rodrigo.guarneros\AppData\Local\Temp\pip-install-mha8smgt\fiona_ab26aab3639e418c96b9756ea5b10a81\
    Complete output (1 lines):
    A GDAL API version must be specified. Provide a path to gdal-config using a GDAL_CONFIG environment variable or use a GDAL_VERSION environment var

In [2]:
!pip install pymongo

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install descartes

Note: you may need to restart the kernel to use updated packages.


In [1]:
# Import dependencies
import pandas as pd
import re
import json
import pymongo
#import geopandas as gpd
import matplotlib.pyplot as plt
import descartes
import os

In [3]:
# Read data
censo = pd.read_csv("Censo_2020_final.csv", encoding='Latin1', low_memory=False)
deposits = pd.read_csv("deposits.csv", encoding='Latin1', low_memory=False)
infrastructure = pd.read_csv("financial_infrastructure.csv", encoding='Latin1', low_memory=False)
use = pd.read_csv("financial_use.csv", encoding='Latin1', low_memory=False)
irs = pd.read_csv("IRS.csv", encoding='Latin1', low_memory=False)


In [4]:
censo.columns

Index(['ï»¿ENTIDAD', 'NOM_ENT', 'MUN', 'NOM_MUN', 'LOC', 'NOM_LOC', 'LONGITUD',
       'LATITUD', 'ALTITUD', 'POBTOT', 'POBFEM', 'POBMAS', 'POB_TOT_6more',
       'POB_TOT_6more_F', 'POB_TOT_6more_M', 'GRAPROES', 'GRAPROES_F',
       'GRAPROES_M', 'POCUPADA', 'POCUPADA_F', 'POCUPADA_M', 'TVIVHAB',
       'VPH_C_ELEC', 'VPH_AGUADV', 'VPH_DRENAJ', 'VPH_REFRI', 'VPH_AUTOM',
       'VPH_MOTO', 'VPH_RADIO', 'VPH_TV', 'VPH_PC', 'VPH_TELEF', 'VPH_CEL',
       'VPH_INTER', 'VPH_STVP', 'VPH_SPMVPI', 'VPH_CVJ', 'TAMLOC',
       'Unnamed: 38', 'Unnamed: 39', 'Unnamed: 40', 'Unnamed: 41',
       'Unnamed: 42', 'Unnamed: 43', 'Unnamed: 44', 'Unnamed: 45',
       'Unnamed: 46', 'Unnamed: 47', 'Unnamed: 48', 'Unnamed: 49',
       'Unnamed: 50'],
      dtype='object')

In [5]:
censo = censo.loc[:, ~censo.columns.str.contains('^Unnamed')]
censo.columns

Index(['ï»¿ENTIDAD', 'NOM_ENT', 'MUN', 'NOM_MUN', 'LOC', 'NOM_LOC', 'LONGITUD',
       'LATITUD', 'ALTITUD', 'POBTOT', 'POBFEM', 'POBMAS', 'POB_TOT_6more',
       'POB_TOT_6more_F', 'POB_TOT_6more_M', 'GRAPROES', 'GRAPROES_F',
       'GRAPROES_M', 'POCUPADA', 'POCUPADA_F', 'POCUPADA_M', 'TVIVHAB',
       'VPH_C_ELEC', 'VPH_AGUADV', 'VPH_DRENAJ', 'VPH_REFRI', 'VPH_AUTOM',
       'VPH_MOTO', 'VPH_RADIO', 'VPH_TV', 'VPH_PC', 'VPH_TELEF', 'VPH_CEL',
       'VPH_INTER', 'VPH_STVP', 'VPH_SPMVPI', 'VPH_CVJ', 'TAMLOC'],
      dtype='object')

In [6]:
censo = censo.rename(columns={'ï»¿ENTIDAD':'Entidad'}, inplace=False)
censo.columns

Index(['Entidad', 'NOM_ENT', 'MUN', 'NOM_MUN', 'LOC', 'NOM_LOC', 'LONGITUD',
       'LATITUD', 'ALTITUD', 'POBTOT', 'POBFEM', 'POBMAS', 'POB_TOT_6more',
       'POB_TOT_6more_F', 'POB_TOT_6more_M', 'GRAPROES', 'GRAPROES_F',
       'GRAPROES_M', 'POCUPADA', 'POCUPADA_F', 'POCUPADA_M', 'TVIVHAB',
       'VPH_C_ELEC', 'VPH_AGUADV', 'VPH_DRENAJ', 'VPH_REFRI', 'VPH_AUTOM',
       'VPH_MOTO', 'VPH_RADIO', 'VPH_TV', 'VPH_PC', 'VPH_TELEF', 'VPH_CEL',
       'VPH_INTER', 'VPH_STVP', 'VPH_SPMVPI', 'VPH_CVJ', 'TAMLOC'],
      dtype='object')

In [7]:
censo

,Entidad,NOM_ENT,MUN,NOM_MUN,LOC,NOM_LOC,LONGITUD,LATITUD,ALTITUD,POBTOT,...,VPH_RADIO,VPH_TV,VPH_PC,VPH_TELEF,VPH_CEL,VPH_INTER,VPH_STVP,VPH_SPMVPI,VPH_CVJ,TAMLOC
0,1,Aguascalientes,1,Aguascalientes,0,Total del Municipio,NaN,NaN,NaN,948990,...,221315,257061,136923,116647,251719,178619,130290,80951,56131,*
1,1,Aguascalientes,1,Aguascalientes,1,Aguascalientes,"102Â°17'45.768"" W","21Â°52'47.362"" N",1878,863893,...,204535,237274,130384,112002,232793,169675,123670,77719,53589,13
2,1,Aguascalientes,1,Aguascalientes,94,Granja Adelita,"102Â°22'24.710"" W","21Â°52'18.749"" N",1902,5,...,*,*,*,*,*,*,*,*,*,1
3,1,Aguascalientes,1,Aguascalientes,96,Agua Azul,"102Â°21'25.639"" W","21Â°53'01.522"" N",1861,41,...,11,11,5,2,11,4,5,2,1,1
4,1,Aguascalientes,1,Aguascalientes,102,Los Arbolitos [Rancho],"102Â°21'26.261"" W","21Â°46'48.650"" N",1861,8,...,*,*,*,*,*,*,*,*,*,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98402,32,Zacatecas,58,Santa MarÃ­a de la Paz,37,Mesa Grande,"103Â°24'52.925"" W","21Â°33'12.375"" N",1944,165,...,29,44,6,29,24,9,13,0,0,1
98403,32,Zacatecas,58,Santa MarÃ­a de la Paz,39,San Isidro,"103Â°20'14.164"" W","21Â°29'11.130"" N",2031,1,...,*,*,*,*,*,*,*,*,*,1
98404,32,Zacatecas,58,Santa MarÃ­a de la Paz,41,San Miguel TepetitlÃ¡n,"103Â°20'09.356"" W","21Â°30'15.168"" N",1977,97,...,20,26,4,1,21,12,21,1,1,1
98405,32,Zacatecas,58,Santa MarÃ­a de la Paz,42,San Rafael,"103Â°22'20.134"" W","21Â°31'39.341"" N",2042,3,...,*,*,*,*,*,*,*,*,*,1


In [16]:
censo['LONGITUD']= censo['LONGITUD'].replace('Â','', regex=True)
censo['LATITUD']= censo['LATITUD'].replace('Â','', regex=True)
censo

,Entidad,NOM_ENT,MUN,NOM_MUN,LOC,NOM_LOC,LONGITUD,LATITUD,ALTITUD,POBTOT,...,VPH_RADIO,VPH_TV,VPH_PC,VPH_TELEF,VPH_CEL,VPH_INTER,VPH_STVP,VPH_SPMVPI,VPH_CVJ,TAMLOC
0,1,Aguascalientes,1,Aguascalientes,0,Total del Municipio,NaN,NaN,NaN,948990,...,221315,257061,136923,116647,251719,178619,130290,80951,56131,*
1,1,Aguascalientes,1,Aguascalientes,1,Aguascalientes,"102°17'45.768"" W","21°52'47.362"" N",1878,863893,...,204535,237274,130384,112002,232793,169675,123670,77719,53589,13
2,1,Aguascalientes,1,Aguascalientes,94,Granja Adelita,"102°22'24.710"" W","21°52'18.749"" N",1902,5,...,*,*,*,*,*,*,*,*,*,1
3,1,Aguascalientes,1,Aguascalientes,96,Agua Azul,"102°21'25.639"" W","21°53'01.522"" N",1861,41,...,11,11,5,2,11,4,5,2,1,1
4,1,Aguascalientes,1,Aguascalientes,102,Los Arbolitos [Rancho],"102°21'26.261"" W","21°46'48.650"" N",1861,8,...,*,*,*,*,*,*,*,*,*,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98402,32,Zacatecas,58,Santa MarÃ­a de la Paz,37,Mesa Grande,"103°24'52.925"" W","21°33'12.375"" N",1944,165,...,29,44,6,29,24,9,13,0,0,1
98403,32,Zacatecas,58,Santa MarÃ­a de la Paz,39,San Isidro,"103°20'14.164"" W","21°29'11.130"" N",2031,1,...,*,*,*,*,*,*,*,*,*,1
98404,32,Zacatecas,58,Santa MarÃ­a de la Paz,41,San Miguel TepetitlÃ¡n,"103°20'09.356"" W","21°30'15.168"" N",1977,97,...,20,26,4,1,21,12,21,1,1,1
98405,32,Zacatecas,58,Santa MarÃ­a de la Paz,42,San Rafael,"103°22'20.134"" W","21°31'39.341"" N",2042,3,...,*,*,*,*,*,*,*,*,*,1


In [8]:
infrastructure

,Clave Municipio,Clave \nEstado,Región,Estado,Municipio,Tipo de población,Branch_BC,Branch_DB,Sucursales Socap,Sucursales Sofipo,Branch_Total,Corresponsales,ATM,PST,Building_PST,MB_contracts
0,1001,1,Occidente y Bajío,Aguascalientes,Aguascalientes,Semi-metrópoli,117,2,16,8,143,458,611,17570,11424,393411
1,1002,1,Occidente y Bajío,Aguascalientes,Asientos,Urbano,0,0,0,0,0,8,4,68,54,8717
2,1003,1,Occidente y Bajío,Aguascalientes,Calvillo,Urbano,5,0,2,0,7,19,17,152,136,11549
3,1004,1,Occidente y Bajío,Aguascalientes,Cosío,Semi-urbano,0,0,0,0,0,5,4,27,21,2044
4,1005,1,Occidente y Bajío,Aguascalientes,Jesús María,Urbano,3,0,1,0,4,48,23,828,654,23794
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2461,32055,32,Occidente y Bajío,Zacatecas,Villanueva,Semi-urbano,3,0,0,0,3,8,3,39,32,4792
2462,32056,32,Occidente y Bajío,Zacatecas,Zacatecas,Urbano,27,1,6,3,37,112,163,3446,2337,93016
2463,32057,32,Occidente y Bajío,Zacatecas,Trancoso,Semi-urbano,0,0,0,0,0,2,2,16,8,2635
2464,32058,32,Occidente y Bajío,Zacatecas,Santa María de la Paz,Rural,0,0,0,0,0,0,0,2,2,77


# Transform
- Census

In [17]:
# First, we remove all the blank values from rows that sum up total, since they don't have any latitude or longitude we can use
censo = censo.dropna()
censo

,Entidad,NOM_ENT,MUN,NOM_MUN,LOC,NOM_LOC,LONGITUD,LATITUD,ALTITUD,POBTOT,...,VPH_RADIO,VPH_TV,VPH_PC,VPH_TELEF,VPH_CEL,VPH_INTER,VPH_STVP,VPH_SPMVPI,VPH_CVJ,TAMLOC
1,1,Aguascalientes,1,Aguascalientes,1,Aguascalientes,"102°17'45.768"" W","21°52'47.362"" N",1878,863893,...,204535,237274,130384,112002,232793,169675,123670,77719,53589,13
2,1,Aguascalientes,1,Aguascalientes,94,Granja Adelita,"102°22'24.710"" W","21°52'18.749"" N",1902,5,...,*,*,*,*,*,*,*,*,*,1
3,1,Aguascalientes,1,Aguascalientes,96,Agua Azul,"102°21'25.639"" W","21°53'01.522"" N",1861,41,...,11,11,5,2,11,4,5,2,1,1
4,1,Aguascalientes,1,Aguascalientes,102,Los Arbolitos [Rancho],"102°21'26.261"" W","21°46'48.650"" N",1861,8,...,*,*,*,*,*,*,*,*,*,1
5,1,Aguascalientes,1,Aguascalientes,104,Ardillas de Abajo (Las Ardillas),"102°11'30.914"" W","21°56'42.243"" N",1989,1,...,*,*,*,*,*,*,*,*,*,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98401,32,Zacatecas,58,Santa MarÃ­a de la Paz,35,Los Trigos (Mesa de los Trigos),"103°14'27.792"" W","21°29'51.355"" N",2499,89,...,13,18,2,0,19,1,5,0,0,1
98402,32,Zacatecas,58,Santa MarÃ­a de la Paz,37,Mesa Grande,"103°24'52.925"" W","21°33'12.375"" N",1944,165,...,29,44,6,29,24,9,13,0,0,1
98403,32,Zacatecas,58,Santa MarÃ­a de la Paz,39,San Isidro,"103°20'14.164"" W","21°29'11.130"" N",2031,1,...,*,*,*,*,*,*,*,*,*,1
98404,32,Zacatecas,58,Santa MarÃ­a de la Paz,41,San Miguel TepetitlÃ¡n,"103°20'09.356"" W","21°30'15.168"" N",1977,97,...,20,26,4,1,21,12,21,1,1,1


In [18]:
# Then, create a function to change the coordinates to the correct type
def dms2dd(s):
    # example: s = """0°51'56.29"S"""
    degrees, minutes, seconds, direction = re.split('[°\'"]+', s)
    dd = float(degrees) + float(minutes)/60 + float(seconds)/(60*60);
    if direction in ('S','W'):
        dd*= -1
    return dd


In [19]:
# We use the function to change both latitude and longitude columns
censo['LATITUD'] = censo['LATITUD'].apply(dms2dd)
censo['LONGITUD'] = censo['LONGITUD'].apply(dms2dd)*-1

<ipython-input-19-e29c7409f113>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  censo['LATITUD'] = censo['LATITUD'].apply(dms2dd)
<ipython-input-19-e29c7409f113>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  censo['LONGITUD'] = censo['LONGITUD'].apply(dms2dd)*-1


In [20]:
censo

,Entidad,NOM_ENT,MUN,NOM_MUN,LOC,NOM_LOC,LONGITUD,LATITUD,ALTITUD,POBTOT,...,VPH_RADIO,VPH_TV,VPH_PC,VPH_TELEF,VPH_CEL,VPH_INTER,VPH_STVP,VPH_SPMVPI,VPH_CVJ,TAMLOC
1,1,Aguascalientes,1,Aguascalientes,1,Aguascalientes,-102.296047,21.879823,1878,863893,...,204535,237274,130384,112002,232793,169675,123670,77719,53589,13
2,1,Aguascalientes,1,Aguascalientes,94,Granja Adelita,-102.373531,21.871875,1902,5,...,*,*,*,*,*,*,*,*,*,1
3,1,Aguascalientes,1,Aguascalientes,96,Agua Azul,-102.357122,21.883756,1861,41,...,11,11,5,2,11,4,5,2,1,1
4,1,Aguascalientes,1,Aguascalientes,102,Los Arbolitos [Rancho],-102.357295,21.780181,1861,8,...,*,*,*,*,*,*,*,*,*,1
5,1,Aguascalientes,1,Aguascalientes,104,Ardillas de Abajo (Las Ardillas),-102.191921,21.945068,1989,1,...,*,*,*,*,*,*,*,*,*,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98401,32,Zacatecas,58,Santa MarÃ­a de la Paz,35,Los Trigos (Mesa de los Trigos),-103.241053,21.497599,2499,89,...,13,18,2,0,19,1,5,0,0,1
98402,32,Zacatecas,58,Santa MarÃ­a de la Paz,37,Mesa Grande,-103.414701,21.553438,1944,165,...,29,44,6,29,24,9,13,0,0,1
98403,32,Zacatecas,58,Santa MarÃ­a de la Paz,39,San Isidro,-103.337268,21.486425,2031,1,...,*,*,*,*,*,*,*,*,*,1
98404,32,Zacatecas,58,Santa MarÃ­a de la Paz,41,San Miguel TepetitlÃ¡n,-103.335932,21.504213,1977,97,...,20,26,4,1,21,12,21,1,1,1


In [25]:
irs = pd.read_csv("IRS.csv", encoding='Latin1', low_memory=False)
irs


,Clave_entidad,Entidad_federativa,Clave_municipio,Municipio,IRS
0,1,Aguascalientes,1001,Aguascalientes,-1.315320
1,1,Aguascalientes,1002,Asientos,-0.857301
2,1,Aguascalientes,1003,Calvillo,-0.918554
3,1,Aguascalientes,1004,Cosío,-1.004023
4,1,Aguascalientes,1005,Jesús María,-1.173361
...,...,...,...,...,...
2464,32,Zacatecas,32054,Villa Hidalgo,-0.692240
2465,32,Zacatecas,32055,Villanueva,-0.884952
2466,32,Zacatecas,32056,Zacatecas,-1.336926
2467,32,Zacatecas,32057,Trancoso,-0.748505


In [34]:
#irs['Clave_municipio2'] = irs['Clave_municipio'].str[-2:]
irs['Clave_municipio'] = irs.Clave_municipio.astype(str).str[2:].astype(int)

In [39]:
irs=irs.drop(['Clave_municipio2', 'Clave_municipio3', 'Clave_municipio4'], axis=1)

In [40]:
irs

,Clave_entidad,Entidad_federativa,Clave_municipio,Municipio,IRS
0,1,Aguascalientes,1,Aguascalientes,-1.315320
1,1,Aguascalientes,2,Asientos,-0.857301
2,1,Aguascalientes,3,Calvillo,-0.918554
3,1,Aguascalientes,4,Cosío,-1.004023
4,1,Aguascalientes,5,Jesús María,-1.173361
...,...,...,...,...,...
2464,32,Zacatecas,54,Villa Hidalgo,-0.692240
2465,32,Zacatecas,55,Villanueva,-0.884952
2466,32,Zacatecas,56,Zacatecas,-1.336926
2467,32,Zacatecas,57,Trancoso,-0.748505


In [43]:
use

,Clave Municipio,Clave \nEstado,Región,Estado,Municipio,Accounts_1,Accounts_2,Accounts_3,Accounts_4,Saving_account,Debit_card,Credit_card,mortgage,Crupal_credit,Personal_credit,payroll_credit,Automotive_credit,ABCD_credit,transactions_pst,transactions_ATM
0,1001,1,Occidente y Bajío,Aguascalientes,Aguascalientes,0,"273,512","6,273","780,665",1,"1,153,278","271,846","21,739","5,365","84,143","48,014","10,474","28,113","2,254,262","1,874,377"
1,1002,1,Occidente y Bajío,Aguascalientes,Asientos,0,"1,454",0,0,0,"1,247","1,122",161,639,"3,028","1,280",102,"1,675","11,265","15,499"
2,1003,1,Occidente y Bajío,Aguascalientes,Calvillo,0,"3,536",22,"18,846",0,"30,233","9,057",103,98,"5,184",919,121,"1,349","10,655","62,460"
3,1004,1,Occidente y Bajío,Aguascalientes,Cosío,0,382,0,0,0,515,436,140,471,"1,746",613,61,951,"4,252","18,031"
4,1005,1,Occidente y Bajío,Aguascalientes,Jesús María,0,"6,430",0,"12,601",0,"21,713","16,352","4,377",671,"8,678","4,476","1,364","5,499","109,775","73,304"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2461,32055,32,Occidente y Bajío,Zacatecas,Villanueva,0,"1,251",43,"8,547",0,"10,494","3,446",25,199,"2,211",534,92,166,"2,664","14,750"
2462,32056,32,Occidente y Bajío,Zacatecas,Zacatecas,0,"38,339","3,377","219,334",0,"395,525","63,976","2,409","2,021","11,152","9,482","2,725","1,988","410,839","380,406"
2463,32057,32,Occidente y Bajío,Zacatecas,Trancoso,0,992,0,0,0,309,216,27,843,631,573,43,59,"4,495","7,135"
2464,32058,32,Occidente y Bajío,Zacatecas,Santa María de la Paz,0,18,0,0,0,10,12,0,0,25,34,4,20,237,0


In [48]:
use['Clave Municipio'] = use['Clave Municipio'].astype(str).str[2:].astype(int)

In [51]:
deposits['Clave Municipio'] = deposits['Clave Municipio'].astype(str).str[2:].astype(int)

,Clave Municipio,Clave \nEstado,Región,Estado,Municipio,Tipo de población,saving_deposits,sight_deposits,term_deposits,debit_deposits
0,1,1,Occidente y Bajío,Aguascalientes,Aguascalientes,Semi-metrópoli,"52,108","91,136","10,440","1,171"
1,2,1,Occidente y Bajío,Aguascalientes,Asientos,Urbano,"1,630","1,067",75,0
2,3,1,Occidente y Bajío,Aguascalientes,Calvillo,Urbano,"6,613","2,974",914,47
3,4,1,Occidente y Bajío,Aguascalientes,Cosío,Semi-urbano,"1,286",724,43,0
4,5,1,Occidente y Bajío,Aguascalientes,Jesús María,Urbano,"8,098","8,160","1,169",65
...,...,...,...,...,...,...,...,...,...,...
2461,55,32,Occidente y Bajío,Zacatecas,Villanueva,Semi-urbano,618,230,49,1
2462,56,32,Occidente y Bajío,Zacatecas,Zacatecas,Urbano,"12,806","12,293","1,072",187
2463,57,32,Occidente y Bajío,Zacatecas,Trancoso,Semi-urbano,470,315,57,0
2464,58,32,Occidente y Bajío,Zacatecas,Santa María de la Paz,Rural,153,76,28,0


In [53]:
infrastructure['Clave Municipio'] = infrastructure['Clave Municipio'].astype(str).str[2:].astype(int)

,Clave Municipio,Clave \nEstado,Región,Estado,Municipio,Tipo de población,Branch_BC,Branch_DB,Sucursales Socap,Sucursales Sofipo,Branch_Total,Corresponsales,ATM,PST,Building_PST,MB_contracts
0,1,1,Occidente y Bajío,Aguascalientes,Aguascalientes,Semi-metrópoli,117,2,16,8,143,458,611,17570,11424,393411
1,2,1,Occidente y Bajío,Aguascalientes,Asientos,Urbano,0,0,0,0,0,8,4,68,54,8717
2,3,1,Occidente y Bajío,Aguascalientes,Calvillo,Urbano,5,0,2,0,7,19,17,152,136,11549
3,4,1,Occidente y Bajío,Aguascalientes,Cosío,Semi-urbano,0,0,0,0,0,5,4,27,21,2044
4,5,1,Occidente y Bajío,Aguascalientes,Jesús María,Urbano,3,0,1,0,4,48,23,828,654,23794
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2461,55,32,Occidente y Bajío,Zacatecas,Villanueva,Semi-urbano,3,0,0,0,3,8,3,39,32,4792
2462,56,32,Occidente y Bajío,Zacatecas,Zacatecas,Urbano,27,1,6,3,37,112,163,3446,2337,93016
2463,57,32,Occidente y Bajío,Zacatecas,Trancoso,Semi-urbano,0,0,0,0,0,2,2,16,8,2635
2464,58,32,Occidente y Bajío,Zacatecas,Santa María de la Paz,Rural,0,0,0,0,0,0,0,2,2,77


In [55]:
censo.columns

Index(['Entidad', 'NOM_ENT', 'MUN', 'NOM_MUN', 'LOC', 'NOM_LOC', 'LONGITUD',
       'LATITUD', 'ALTITUD', 'POBTOT', 'POBFEM', 'POBMAS', 'POB_TOT_6more',
       'POB_TOT_6more_F', 'POB_TOT_6more_M', 'GRAPROES', 'GRAPROES_F',
       'GRAPROES_M', 'POCUPADA', 'POCUPADA_F', 'POCUPADA_M', 'TVIVHAB',
       'VPH_C_ELEC', 'VPH_AGUADV', 'VPH_DRENAJ', 'VPH_REFRI', 'VPH_AUTOM',
       'VPH_MOTO', 'VPH_RADIO', 'VPH_TV', 'VPH_PC', 'VPH_TELEF', 'VPH_CEL',
       'VPH_INTER', 'VPH_STVP', 'VPH_SPMVPI', 'VPH_CVJ', 'TAMLOC'],
      dtype='object')

In [56]:
use.columns

Index(['Clave Municipio', 'Clave \nEstado', 'Región', 'Estado', 'Municipio',
       'Accounts_1', 'Accounts_2', 'Accounts_3', 'Accounts_4',
       'Saving_account', 'Debit_card', 'Credit_card', 'mortgage',
       'Crupal_credit', 'Personal_credit', 'payroll_credit',
       'Automotive_credit', 'ABCD_credit', 'transactions_pst',
       'transactions_ATM'],
      dtype='object')

In [57]:
infrastructure.columns

Index(['Clave Municipio', 'Clave \nEstado', 'Región', 'Estado', 'Municipio',
       'Tipo de población', 'Branch_BC', 'Branch_DB', 'Sucursales Socap',
       'Sucursales Sofipo', 'Branch_Total', 'Corresponsales', 'ATM', 'PST',
       'Building_PST', 'MB_contracts'],
      dtype='object')

In [59]:
deposits.columns

Index(['Clave Municipio', 'Clave \nEstado', 'Región', 'Estado', 'Municipio',
       'Tipo de población', 'saving_deposits', 'sight_deposits',
       'term_deposits', 'debit_deposits'],
      dtype='object')

In [60]:
deposits.describe

<bound method NDFrame.describe of       Clave Municipio  Clave \nEstado             Región           Estado  \
0                   1               1  Occidente y Bajío   Aguascalientes   
1                   2               1  Occidente y Bajío   Aguascalientes   
2                   3               1  Occidente y Bajío   Aguascalientes   
3                   4               1  Occidente y Bajío   Aguascalientes   
4                   5               1  Occidente y Bajío   Aguascalientes   
...               ...             ...                ...              ...   
2461               55              32  Occidente y Bajío        Zacatecas   
2462               56              32  Occidente y Bajío        Zacatecas   
2463               57              32  Occidente y Bajío        Zacatecas   
2464               58              32  Occidente y Bajío        Zacatecas   
2465              999              99    Sin identificar  Sin identificar   

                  Municipio Tipo de pobla

In [61]:
infrastructure.describe

<bound method NDFrame.describe of       Clave Municipio  Clave \nEstado             Región           Estado  \
0                   1               1  Occidente y Bajío   Aguascalientes   
1                   2               1  Occidente y Bajío   Aguascalientes   
2                   3               1  Occidente y Bajío   Aguascalientes   
3                   4               1  Occidente y Bajío   Aguascalientes   
4                   5               1  Occidente y Bajío   Aguascalientes   
...               ...             ...                ...              ...   
2461               55              32  Occidente y Bajío        Zacatecas   
2462               56              32  Occidente y Bajío        Zacatecas   
2463               57              32  Occidente y Bajío        Zacatecas   
2464               58              32  Occidente y Bajío        Zacatecas   
2465              999              99    Sin identificar  Sin identificar   

                  Municipio Tipo de pobla

In [63]:
censo.columns

Index(['Entidad', 'NOM_ENT', 'MUN', 'NOM_MUN', 'LOC', 'NOM_LOC', 'LONGITUD',
       'LATITUD', 'ALTITUD', 'POBTOT', 'POBFEM', 'POBMAS', 'POB_TOT_6more',
       'POB_TOT_6more_F', 'POB_TOT_6more_M', 'GRAPROES', 'GRAPROES_F',
       'GRAPROES_M', 'POCUPADA', 'POCUPADA_F', 'POCUPADA_M', 'TVIVHAB',
       'VPH_C_ELEC', 'VPH_AGUADV', 'VPH_DRENAJ', 'VPH_REFRI', 'VPH_AUTOM',
       'VPH_MOTO', 'VPH_RADIO', 'VPH_TV', 'VPH_PC', 'VPH_TELEF', 'VPH_CEL',
       'VPH_INTER', 'VPH_STVP', 'VPH_SPMVPI', 'VPH_CVJ', 'TAMLOC'],
      dtype='object')

In [69]:
censo = censo.drop(['LOC', 'NOM_LOC', 'LONGITUD', 'LATITUD', 'ALTITUD'], axis=1)

In [74]:
Columns = ['POBTOT', 'POBFEM', 'POBMAS', 'POB_TOT_6more',
       'POB_TOT_6more_F', 'POB_TOT_6more_M', 'GRAPROES', 'GRAPROES_F',
       'GRAPROES_M', 'POCUPADA', 'POCUPADA_F', 'POCUPADA_M', 'TVIVHAB',
       'VPH_C_ELEC', 'VPH_AGUADV', 'VPH_DRENAJ', 'VPH_REFRI', 'VPH_AUTOM',
       'VPH_MOTO', 'VPH_RADIO', 'VPH_TV', 'VPH_PC', 'VPH_TELEF', 'VPH_CEL',
       'VPH_INTER', 'VPH_STVP', 'VPH_SPMVPI', 'VPH_CVJ', 'TAMLOC']

In [76]:
censo[Columns] = censo[Columns].apply(pd.to_numeric, errors = 'coerce')
censo

,Entidad,NOM_ENT,MUN,NOM_MUN,POBTOT,POBFEM,POBMAS,POB_TOT_6more,POB_TOT_6more_F,POB_TOT_6more_M,...,VPH_RADIO,VPH_TV,VPH_PC,VPH_TELEF,VPH_CEL,VPH_INTER,VPH_STVP,VPH_SPMVPI,VPH_CVJ,TAMLOC
1,1,Aguascalientes,1,Aguascalientes,863893,444725.0,419168.0,722445.0,482914.0,239531.0,...,204535.0,237274.0,130384.0,112002.0,232793.0,169675.0,123670.0,77719.0,53589.0,13.0
2,1,Aguascalientes,1,Aguascalientes,5,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
3,1,Aguascalientes,1,Aguascalientes,41,17.0,24.0,31.0,19.0,12.0,...,11.0,11.0,5.0,2.0,11.0,4.0,5.0,2.0,1.0,1.0
4,1,Aguascalientes,1,Aguascalientes,8,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
5,1,Aguascalientes,1,Aguascalientes,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98401,32,Zacatecas,58,Santa MarÃ­a de la Paz,89,43.0,46.0,70.0,42.0,28.0,...,13.0,18.0,2.0,0.0,19.0,1.0,5.0,0.0,0.0,1.0
98402,32,Zacatecas,58,Santa MarÃ­a de la Paz,165,88.0,77.0,156.0,101.0,55.0,...,29.0,44.0,6.0,29.0,24.0,9.0,13.0,0.0,0.0,1.0
98403,32,Zacatecas,58,Santa MarÃ­a de la Paz,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
98404,32,Zacatecas,58,Santa MarÃ­a de la Paz,97,50.0,47.0,87.0,53.0,34.0,...,20.0,26.0,4.0,1.0,21.0,12.0,21.0,1.0,1.0,1.0


In [97]:
censo_df = censo.groupby(['Entidad',"MUN", 'NOM_ENT', 'NOM_MUN']).sum().reset_index()
censo_df

,Entidad,MUN,NOM_ENT,NOM_MUN,POBTOT,POBFEM,POBMAS,POB_TOT_6more,POB_TOT_6more_F,POB_TOT_6more_M,...,VPH_RADIO,VPH_TV,VPH_PC,VPH_TELEF,VPH_CEL,VPH_INTER,VPH_STVP,VPH_SPMVPI,VPH_CVJ,TAMLOC
0,1,1,Aguascalientes,Aguascalientes,919739,472763.0,446719.0,771121.0,514631.0,256508.0,...,215368.0,250028.0,133644.0,114196.0,244819.0,174765.0,127205.0,78731.0,54547.0,274.0
1,16,15,Michoacán de Ocampo,CoalcomÃ¡n de VÃ¡zquez Pallares,3711,1316.0,1459.0,2387.0,1474.0,913.0,...,431.0,327.0,32.0,10.0,479.0,286.0,241.0,16.0,12.0,256.0
2,16,16,Michoacán de Ocampo,Coeneo,41943,22077.0,19840.0,35843.0,23432.0,12411.0,...,7502.0,10861.0,1854.0,1401.0,9788.0,2448.0,6584.0,358.0,352.0,81.0
3,16,17,Michoacán de Ocampo,Contepec,70159,36252.0,33888.0,61144.0,39840.0,21304.0,...,11962.0,16648.0,2012.0,1600.0,15188.0,2594.0,3740.0,340.0,408.0,147.0
4,16,18,Michoacán de Ocampo,CopÃ¡ndaro,18968,9780.0,9182.0,16538.0,10668.0,5870.0,...,3340.0,4588.0,648.0,438.0,4146.0,1110.0,1664.0,120.0,122.0,32.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1678,32,54,Zacatecas,Villa Hidalgo,19446,9900.0,9458.0,16786.0,10817.0,5969.0,...,2901.0,4652.0,662.0,909.0,3913.0,1148.0,1087.0,109.0,106.0,84.0
1679,32,55,Zacatecas,Villanueva,31558,15865.0,15415.0,26875.0,17158.0,9717.0,...,5941.0,8382.0,2201.0,3163.0,7367.0,4402.0,5161.0,642.0,567.0,153.0
1680,32,56,Zacatecas,Zacatecas,149607,77547.0,71875.0,124256.0,83309.0,40947.0,...,33117.0,40791.0,24097.0,20493.0,39717.0,31982.0,28619.0,11941.0,7639.0,105.0
1681,32,57,Zacatecas,Trancoso,20455,10387.0,10003.0,17623.0,11460.0,6163.0,...,3432.0,4390.0,864.0,948.0,3819.0,1089.0,1848.0,202.0,184.0,37.0


In [98]:
censo_df['NOM_ENT']= censo_df['NOM_ENT'].replace('Â¡','á', regex=True)
censo_df['NOM_ENT']= censo_df['NOM_ENT'].replace('Ã³','ó', regex=True)
censo_df['NOM_ENT']= censo_df['NOM_ENT'].replace('Ã©','é', regex=True)
censo_df['NOM_ENT']= censo_df['NOM_ENT'].replace('Ã\xad','í', regex=True)
censo_df['NOM_ENT']= censo_df['NOM_ENT'].replace('Ã¡','á', regex=True)
censo_df

,Entidad,MUN,NOM_ENT,NOM_MUN,POBTOT,POBFEM,POBMAS,POB_TOT_6more,POB_TOT_6more_F,POB_TOT_6more_M,...,VPH_RADIO,VPH_TV,VPH_PC,VPH_TELEF,VPH_CEL,VPH_INTER,VPH_STVP,VPH_SPMVPI,VPH_CVJ,TAMLOC
0,1,1,Aguascalientes,Aguascalientes,919739,472763.0,446719.0,771121.0,514631.0,256508.0,...,215368.0,250028.0,133644.0,114196.0,244819.0,174765.0,127205.0,78731.0,54547.0,274.0
1,16,15,Michoacán de Ocampo,CoalcomÃ¡n de VÃ¡zquez Pallares,3711,1316.0,1459.0,2387.0,1474.0,913.0,...,431.0,327.0,32.0,10.0,479.0,286.0,241.0,16.0,12.0,256.0
2,16,16,Michoacán de Ocampo,Coeneo,41943,22077.0,19840.0,35843.0,23432.0,12411.0,...,7502.0,10861.0,1854.0,1401.0,9788.0,2448.0,6584.0,358.0,352.0,81.0
3,16,17,Michoacán de Ocampo,Contepec,70159,36252.0,33888.0,61144.0,39840.0,21304.0,...,11962.0,16648.0,2012.0,1600.0,15188.0,2594.0,3740.0,340.0,408.0,147.0
4,16,18,Michoacán de Ocampo,CopÃ¡ndaro,18968,9780.0,9182.0,16538.0,10668.0,5870.0,...,3340.0,4588.0,648.0,438.0,4146.0,1110.0,1664.0,120.0,122.0,32.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1678,32,54,Zacatecas,Villa Hidalgo,19446,9900.0,9458.0,16786.0,10817.0,5969.0,...,2901.0,4652.0,662.0,909.0,3913.0,1148.0,1087.0,109.0,106.0,84.0
1679,32,55,Zacatecas,Villanueva,31558,15865.0,15415.0,26875.0,17158.0,9717.0,...,5941.0,8382.0,2201.0,3163.0,7367.0,4402.0,5161.0,642.0,567.0,153.0
1680,32,56,Zacatecas,Zacatecas,149607,77547.0,71875.0,124256.0,83309.0,40947.0,...,33117.0,40791.0,24097.0,20493.0,39717.0,31982.0,28619.0,11941.0,7639.0,105.0
1681,32,57,Zacatecas,Trancoso,20455,10387.0,10003.0,17623.0,11460.0,6163.0,...,3432.0,4390.0,864.0,948.0,3819.0,1089.0,1848.0,202.0,184.0,37.0


In [99]:
censo_df['NOM_ENT'].unique()

array(['Aguascalientes', 'Michoacán de Ocampo', 'Morelos', 'Nayarit',
       'Nuevo León', 'Oaxaca', 'Puebla', 'Querétaro', 'Quintana Roo',
       'San Luis Potosí', 'Sinaloa', 'Sonora', 'Tabasco', 'Tamaulipas',
       'Tlaxcala', 'Veracruz de Ignacio de la Llave', 'Yucatán',
       'Zacatecas'], dtype=object)

In [100]:
censo_df['NOM_MUN']= censo_df['NOM_MUN'].replace('Â¡','á', regex=True)
censo_df['NOM_MUN']= censo_df['NOM_MUN'].replace('Ã³','ó', regex=True)
censo_df['NOM_MUN']= censo_df['NOM_MUN'].replace('Ã©','é', regex=True)
censo_df['NOM_MUN']= censo_df['NOM_MUN'].replace('Ã\xad','í', regex=True)
censo_df['NOM_MUN']= censo_df['NOM_MUN'].replace('Ã¡','á', regex=True)
censo_df

,Entidad,MUN,NOM_ENT,NOM_MUN,POBTOT,POBFEM,POBMAS,POB_TOT_6more,POB_TOT_6more_F,POB_TOT_6more_M,...,VPH_RADIO,VPH_TV,VPH_PC,VPH_TELEF,VPH_CEL,VPH_INTER,VPH_STVP,VPH_SPMVPI,VPH_CVJ,TAMLOC
0,1,1,Aguascalientes,Aguascalientes,919739,472763.0,446719.0,771121.0,514631.0,256508.0,...,215368.0,250028.0,133644.0,114196.0,244819.0,174765.0,127205.0,78731.0,54547.0,274.0
1,16,15,Michoacán de Ocampo,Coalcomán de Vázquez Pallares,3711,1316.0,1459.0,2387.0,1474.0,913.0,...,431.0,327.0,32.0,10.0,479.0,286.0,241.0,16.0,12.0,256.0
2,16,16,Michoacán de Ocampo,Coeneo,41943,22077.0,19840.0,35843.0,23432.0,12411.0,...,7502.0,10861.0,1854.0,1401.0,9788.0,2448.0,6584.0,358.0,352.0,81.0
3,16,17,Michoacán de Ocampo,Contepec,70159,36252.0,33888.0,61144.0,39840.0,21304.0,...,11962.0,16648.0,2012.0,1600.0,15188.0,2594.0,3740.0,340.0,408.0,147.0
4,16,18,Michoacán de Ocampo,Copándaro,18968,9780.0,9182.0,16538.0,10668.0,5870.0,...,3340.0,4588.0,648.0,438.0,4146.0,1110.0,1664.0,120.0,122.0,32.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1678,32,54,Zacatecas,Villa Hidalgo,19446,9900.0,9458.0,16786.0,10817.0,5969.0,...,2901.0,4652.0,662.0,909.0,3913.0,1148.0,1087.0,109.0,106.0,84.0
1679,32,55,Zacatecas,Villanueva,31558,15865.0,15415.0,26875.0,17158.0,9717.0,...,5941.0,8382.0,2201.0,3163.0,7367.0,4402.0,5161.0,642.0,567.0,153.0
1680,32,56,Zacatecas,Zacatecas,149607,77547.0,71875.0,124256.0,83309.0,40947.0,...,33117.0,40791.0,24097.0,20493.0,39717.0,31982.0,28619.0,11941.0,7639.0,105.0
1681,32,57,Zacatecas,Trancoso,20455,10387.0,10003.0,17623.0,11460.0,6163.0,...,3432.0,4390.0,864.0,948.0,3819.0,1089.0,1848.0,202.0,184.0,37.0


In [101]:
censo_df['NOM_MUN'].unique()


array(['Aguascalientes', 'Coalcomán de Vázquez Pallares', 'Coeneo', ...,
       'Zacatecas', 'Trancoso', 'Santa María de la Paz'], dtype=object)

In [92]:
deposits['Municipio'].unique()

array(['Aguascalientes', 'Asientos', 'Calvillo', ..., 'Trancoso',
       'Santa María de la Paz', 'Sin identificar'], dtype=object)

In [102]:
censo_df.columns

Index(['Entidad', 'MUN', 'NOM_ENT', 'NOM_MUN', 'POBTOT', 'POBFEM', 'POBMAS',
       'POB_TOT_6more', 'POB_TOT_6more_F', 'POB_TOT_6more_M', 'GRAPROES',
       'GRAPROES_F', 'GRAPROES_M', 'POCUPADA', 'POCUPADA_F', 'POCUPADA_M',
       'TVIVHAB', 'VPH_C_ELEC', 'VPH_AGUADV', 'VPH_DRENAJ', 'VPH_REFRI',
       'VPH_AUTOM', 'VPH_MOTO', 'VPH_RADIO', 'VPH_TV', 'VPH_PC', 'VPH_TELEF',
       'VPH_CEL', 'VPH_INTER', 'VPH_STVP', 'VPH_SPMVPI', 'VPH_CVJ', 'TAMLOC'],
      dtype='object')

In [103]:
infrastructure.columns

Index(['Clave Municipio', 'Clave \nEstado', 'Región', 'Estado', 'Municipio',
       'Tipo de población', 'Branch_BC', 'Branch_DB', 'Sucursales Socap',
       'Sucursales Sofipo', 'Branch_Total', 'Corresponsales', 'ATM', 'PST',
       'Building_PST', 'MB_contracts'],
      dtype='object')

In [104]:
merge = censo_df.merge(infrastructure,how='left', left_on=['Entidad','MUN'],right_on=['Clave \nEstado','Clave Municipio'])


In [105]:
merge

,Entidad,MUN,NOM_ENT,NOM_MUN,POBTOT,POBFEM,POBMAS,POB_TOT_6more,POB_TOT_6more_F,POB_TOT_6more_M,...,Branch_BC,Branch_DB,Sucursales Socap,Sucursales Sofipo,Branch_Total,Corresponsales,ATM,PST,Building_PST,MB_contracts
0,1,1,Aguascalientes,Aguascalientes,919739,472763.0,446719.0,771121.0,514631.0,256508.0,...,117.0,2.0,16.0,8.0,143.0,458.0,611.0,17570.0,11424.0,393411.0
1,16,15,Michoacán de Ocampo,Coalcomán de Vázquez Pallares,3711,1316.0,1459.0,2387.0,1474.0,913.0,...,2.0,0.0,0.0,0.0,2.0,3.0,6.0,49.0,43.0,7365.0
2,16,16,Michoacán de Ocampo,Coeneo,41943,22077.0,19840.0,35843.0,23432.0,12411.0,...,1.0,1.0,1.0,0.0,3.0,2.0,1.0,39.0,35.0,2680.0
3,16,17,Michoacán de Ocampo,Contepec,70159,36252.0,33888.0,61144.0,39840.0,21304.0,...,1.0,0.0,2.0,0.0,3.0,3.0,3.0,82.0,51.0,3713.0
4,16,18,Michoacán de Ocampo,Copándaro,18968,9780.0,9182.0,16538.0,10668.0,5870.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,15.0,9.0,940.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1678,32,54,Zacatecas,Villa Hidalgo,19446,9900.0,9458.0,16786.0,10817.0,5969.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,15.0,11.0,1303.0
1679,32,55,Zacatecas,Villanueva,31558,15865.0,15415.0,26875.0,17158.0,9717.0,...,3.0,0.0,0.0,0.0,3.0,8.0,3.0,39.0,32.0,4792.0
1680,32,56,Zacatecas,Zacatecas,149607,77547.0,71875.0,124256.0,83309.0,40947.0,...,27.0,1.0,6.0,3.0,37.0,112.0,163.0,3446.0,2337.0,93016.0
1681,32,57,Zacatecas,Trancoso,20455,10387.0,10003.0,17623.0,11460.0,6163.0,...,0.0,0.0,0.0,0.0,0.0,2.0,2.0,16.0,8.0,2635.0


In [106]:
merge.columns

Index(['Entidad', 'MUN', 'NOM_ENT', 'NOM_MUN', 'POBTOT', 'POBFEM', 'POBMAS',
       'POB_TOT_6more', 'POB_TOT_6more_F', 'POB_TOT_6more_M', 'GRAPROES',
       'GRAPROES_F', 'GRAPROES_M', 'POCUPADA', 'POCUPADA_F', 'POCUPADA_M',
       'TVIVHAB', 'VPH_C_ELEC', 'VPH_AGUADV', 'VPH_DRENAJ', 'VPH_REFRI',
       'VPH_AUTOM', 'VPH_MOTO', 'VPH_RADIO', 'VPH_TV', 'VPH_PC', 'VPH_TELEF',
       'VPH_CEL', 'VPH_INTER', 'VPH_STVP', 'VPH_SPMVPI', 'VPH_CVJ', 'TAMLOC',
       'Clave Municipio', 'Clave \nEstado', 'Región', 'Estado', 'Municipio',
       'Tipo de población', 'Branch_BC', 'Branch_DB', 'Sucursales Socap',
       'Sucursales Sofipo', 'Branch_Total', 'Corresponsales', 'ATM', 'PST',
       'Building_PST', 'MB_contracts'],
      dtype='object')

In [107]:
merge2 = merge.merge(deposits,how='left', left_on=['Entidad','MUN'],right_on=['Clave \nEstado','Clave Municipio'])

In [109]:
merge2.columns

Index(['Entidad', 'MUN', 'NOM_ENT', 'NOM_MUN', 'POBTOT', 'POBFEM', 'POBMAS',
       'POB_TOT_6more', 'POB_TOT_6more_F', 'POB_TOT_6more_M', 'GRAPROES',
       'GRAPROES_F', 'GRAPROES_M', 'POCUPADA', 'POCUPADA_F', 'POCUPADA_M',
       'TVIVHAB', 'VPH_C_ELEC', 'VPH_AGUADV', 'VPH_DRENAJ', 'VPH_REFRI',
       'VPH_AUTOM', 'VPH_MOTO', 'VPH_RADIO', 'VPH_TV', 'VPH_PC', 'VPH_TELEF',
       'VPH_CEL', 'VPH_INTER', 'VPH_STVP', 'VPH_SPMVPI', 'VPH_CVJ', 'TAMLOC',
       'Clave Municipio_x', 'Clave \nEstado_x', 'Región_x', 'Estado_x',
       'Municipio_x', 'Tipo de población_x', 'Branch_BC', 'Branch_DB',
       'Sucursales Socap', 'Sucursales Sofipo', 'Branch_Total',
       'Corresponsales', 'ATM', 'PST', 'Building_PST', 'MB_contracts',
       'Clave Municipio_y', 'Clave \nEstado_y', 'Región_y', 'Estado_y',
       'Municipio_y', 'Tipo de población_y', 'saving_deposits',
       'sight_deposits', 'term_deposits', 'debit_deposits'],
      dtype='object')

In [110]:
merge3 = merge2.merge(use,how='left', left_on=['Entidad','MUN'],right_on=['Clave \nEstado','Clave Municipio'])
merge3

,Entidad,MUN,NOM_ENT,NOM_MUN,POBTOT,POBFEM,POBMAS,POB_TOT_6more,POB_TOT_6more_F,POB_TOT_6more_M,...,Debit_card,Credit_card,mortgage,Crupal_credit,Personal_credit,payroll_credit,Automotive_credit,ABCD_credit,transactions_pst,transactions_ATM
0,1,1,Aguascalientes,Aguascalientes,919739,472763.0,446719.0,771121.0,514631.0,256508.0,...,"1,153,278","271,846","21,739","5,365","84,143","48,014","10,474","28,113","2,254,262","1,874,377"
1,16,15,Michoacán de Ocampo,Coalcomán de Vázquez Pallares,3711,1316.0,1459.0,2387.0,1474.0,913.0,...,"11,037","2,116",10,156,"2,469",389,27,111,"5,353","18,007"
2,16,16,Michoacán de Ocampo,Coeneo,41943,22077.0,19840.0,35843.0,23432.0,12411.0,...,"11,626",621,4,173,"1,600",176,15,319,"1,323",954
3,16,17,Michoacán de Ocampo,Contepec,70159,36252.0,33888.0,61144.0,39840.0,21304.0,...,"2,619",476,44,460,"3,298",331,27,"1,095","7,874","2,090"
4,16,18,Michoacán de Ocampo,Copándaro,18968,9780.0,9182.0,16538.0,10668.0,5870.0,...,71,106,1,123,380,79,3,77,"6,343","1,271"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1678,32,54,Zacatecas,Villa Hidalgo,19446,9900.0,9458.0,16786.0,10817.0,5969.0,...,65,137,8,142,266,261,28,246,"1,565","5,929"
1679,32,55,Zacatecas,Villanueva,31558,15865.0,15415.0,26875.0,17158.0,9717.0,...,"10,494","3,446",25,199,"2,211",534,92,166,"2,664","14,750"
1680,32,56,Zacatecas,Zacatecas,149607,77547.0,71875.0,124256.0,83309.0,40947.0,...,"395,525","63,976","2,409","2,021","11,152","9,482","2,725","1,988","410,839","380,406"
1681,32,57,Zacatecas,Trancoso,20455,10387.0,10003.0,17623.0,11460.0,6163.0,...,309,216,27,843,631,573,43,59,"4,495","7,135"


In [117]:
irs['Clave_entidad'] = irs['Clave_entidad'].apply(pd.to_numeric, errors = 'coerce')
irs['Clave_municipio'] = irs['Clave_municipio'].apply(pd.to_numeric, errors = 'coerce')
irs.dtypes

Clave_entidad           int64
Entidad_federativa     object
Clave_municipio         int64
Municipio              object
IRS                   float64
dtype: object

In [119]:
merge4 = merge3.merge(irs,how='left', left_on=['Entidad','MUN'],right_on=['Clave_entidad','Clave_municipio'])

In [126]:
merge4

,Entidad,MUN,NOM_ENT,NOM_MUN,POBTOT,POBFEM,POBMAS,POB_TOT_6more,POB_TOT_6more_F,POB_TOT_6more_M,...,payroll_credit,Automotive_credit,ABCD_credit,transactions_pst,transactions_ATM,Clave_entidad,Entidad_federativa,Clave_municipio,Municipio_y,IRS
0,1,1,Aguascalientes,Aguascalientes,919739,472763.0,446719.0,771121.0,514631.0,256508.0,...,"48,014","10,474","28,113","2,254,262","1,874,377",1.0,Aguascalientes,1.0,Aguascalientes,-1.315320
1,16,15,Michoacán de Ocampo,Coalcomán de Vázquez Pallares,3711,1316.0,1459.0,2387.0,1474.0,913.0,...,389,27,111,"5,353","18,007",16.0,Michoacán de Ocampo,15.0,Coalcomán de Vázquez Pallares,-0.268599
2,16,16,Michoacán de Ocampo,Coeneo,41943,22077.0,19840.0,35843.0,23432.0,12411.0,...,176,15,319,"1,323",954,16.0,Michoacán de Ocampo,16.0,Coeneo,-0.367754
3,16,17,Michoacán de Ocampo,Contepec,70159,36252.0,33888.0,61144.0,39840.0,21304.0,...,331,27,"1,095","7,874","2,090",16.0,Michoacán de Ocampo,17.0,Contepec,-0.287302
4,16,18,Michoacán de Ocampo,Copándaro,18968,9780.0,9182.0,16538.0,10668.0,5870.0,...,79,3,77,"6,343","1,271",16.0,Michoacán de Ocampo,18.0,Copándaro,-0.579551
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1678,32,54,Zacatecas,Villa Hidalgo,19446,9900.0,9458.0,16786.0,10817.0,5969.0,...,261,28,246,"1,565","5,929",32.0,Zacatecas,54.0,Villa Hidalgo,-0.692240
1679,32,55,Zacatecas,Villanueva,31558,15865.0,15415.0,26875.0,17158.0,9717.0,...,534,92,166,"2,664","14,750",32.0,Zacatecas,55.0,Villanueva,-0.884952
1680,32,56,Zacatecas,Zacatecas,149607,77547.0,71875.0,124256.0,83309.0,40947.0,...,"9,482","2,725","1,988","410,839","380,406",32.0,Zacatecas,56.0,Zacatecas,-1.336926
1681,32,57,Zacatecas,Trancoso,20455,10387.0,10003.0,17623.0,11460.0,6163.0,...,573,43,59,"4,495","7,135",32.0,Zacatecas,57.0,Trancoso,-0.748505


In [133]:
merge4 = merge4.reset_index()

In [137]:
merge4.columns

Index(['index', 'Entidad', 'MUN', 'NOM_ENT', 'NOM_MUN', 'POBTOT', 'POBFEM',
       'POBMAS', 'POB_TOT_6more', 'POB_TOT_6more_F', 'POB_TOT_6more_M',
       'GRAPROES', 'GRAPROES_F', 'GRAPROES_M', 'POCUPADA', 'POCUPADA_F',
       'POCUPADA_M', 'TVIVHAB', 'VPH_C_ELEC', 'VPH_AGUADV', 'VPH_DRENAJ',
       'VPH_REFRI', 'VPH_AUTOM', 'VPH_MOTO', 'VPH_RADIO', 'VPH_TV', 'VPH_PC',
       'VPH_TELEF', 'VPH_CEL', 'VPH_INTER', 'VPH_STVP', 'VPH_SPMVPI',
       'VPH_CVJ', 'TAMLOC', 'Clave Municipio_x', 'Clave \nEstado_x',
       'Región_x', 'Estado_x', 'Municipio_x', 'Tipo de población_x',
       'Branch_BC', 'Branch_DB', 'Sucursales Socap', 'Sucursales Sofipo',
       'Branch_Total', 'Corresponsales', 'ATM', 'PST', 'Building_PST',
       'MB_contracts', 'Clave Municipio_y', 'Clave \nEstado_y', 'Región_y',
       'Estado_y', 'Municipio_y', 'Tipo de población_y', 'saving_deposits',
       'sight_deposits', 'term_deposits', 'debit_deposits', 'Clave Municipio',
       'Clave \nEstado', 'Región', 'Estado'

In [139]:
merge4.drop(['Clave Municipio_x', 'Clave \nEstado_x', 'Región_x', 'Estado_x', 'Municipio_x', 'Tipo de población_x'], axis='columns', inplace=True)

In [140]:
merge4.columns

Index(['index', 'Entidad', 'MUN', 'NOM_ENT', 'NOM_MUN', 'POBTOT', 'POBFEM',
       'POBMAS', 'POB_TOT_6more', 'POB_TOT_6more_F', 'POB_TOT_6more_M',
       'GRAPROES', 'GRAPROES_F', 'GRAPROES_M', 'POCUPADA', 'POCUPADA_F',
       'POCUPADA_M', 'TVIVHAB', 'VPH_C_ELEC', 'VPH_AGUADV', 'VPH_DRENAJ',
       'VPH_REFRI', 'VPH_AUTOM', 'VPH_MOTO', 'VPH_RADIO', 'VPH_TV', 'VPH_PC',
       'VPH_TELEF', 'VPH_CEL', 'VPH_INTER', 'VPH_STVP', 'VPH_SPMVPI',
       'VPH_CVJ', 'TAMLOC', 'Branch_BC', 'Branch_DB', 'Sucursales Socap',
       'Sucursales Sofipo', 'Branch_Total', 'Corresponsales', 'ATM', 'PST',
       'Building_PST', 'MB_contracts', 'Clave Municipio_y', 'Clave \nEstado_y',
       'Región_y', 'Estado_y', 'Municipio_y', 'Tipo de población_y',
       'saving_deposits', 'sight_deposits', 'term_deposits', 'debit_deposits',
       'Clave Municipio', 'Clave \nEstado', 'Región', 'Estado', 'Accounts_1',
       'Accounts_2', 'Accounts_3', 'Accounts_4', 'Saving_account',
       'Debit_card', 'Credit_card'